In [1]:
import geopandas as gpd, folium, branca
from folium.plugins import Search

In [2]:
folium.__version__

'0.6.0+23.ga044e82'

In [3]:
states = gpd.read_file(r"https://rawcdn.githack.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json")

cities = gpd.read_file(r"https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_populated_places_simple.geojson", driver='GeoJSON')

In [4]:
states.head(3)

,id,name,density,geometry
0,01,Alabama,94.650,"POLYGON ((-87.359296 35.00118, -85.606675 34.9..."
1,02,Alaska,1.264,"(POLYGON ((-131.602021 55.117982, -131.569159 ..."
2,04,Arizona,57.050,"POLYGON ((-109.042503 37.000263, -109.04798 31..."


In [5]:
cities.head(3)

,scalerank,natscale,labelrank,name,namepar,namealt,diffascii,nameascii,adm0cap,capalt,...,pop_other,rank_max,rank_min,geonameid,meganame,ls_name,ls_match,checkme,featureclass,geometry
0,10,1,5,Bombo,None,None,0,Bombo,0,0,...,0,8,7,-1,None,None,0,0,Admin-1 region capital,POINT (32.53329952486484 0.5832991056146281)
1,10,1,5,Fort Portal,None,None,0,Fort Portal,0,0,...,0,7,7,233476,None,None,0,0,Admin-1 region capital,POINT (30.27500161597942 0.671004121125236)
2,10,1,3,Potenza,None,None,0,Potenza,0,0,...,0,8,8,3170027,None,None,0,0,Admin-1 region capital,POINT (15.79899649564027 40.64200213009821)


In [6]:
us_cities = gpd.sjoin(cities, states, how='inner', op='within')
us_cities.head(3)

,scalerank,natscale,labelrank,name_left,namepar,namealt,diffascii,nameascii,adm0cap,capalt,...,meganame,ls_name,ls_match,checkme,featureclass,geometry,index_right,id,name_right,density
123,4,50,1,San Bernardino,None,Riverside-San Bernardino,0,San Bernardino,0,0,...,Riverside-San Bernardino,San Bernardino,1,0,Populated place,POINT (-117.3019800808744 34.1223295856538),4,06,California,241.7
172,4,50,1,Santa Barbara,None,None,0,Santa Barbara,0,0,...,None,Santa Barbara2,1,0,Populated place,POINT (-119.7199898700576 34.43498984683384),4,06,California,241.7
173,4,50,1,Fresno,None,None,0,Fresno,0,0,...,None,Fresno,1,0,Populated place,POINT (-119.7729840513729 36.74771689762446),4,06,California,241.7


In [7]:
colormap = branca.colormap.StepColormap(colors=['green','yellow','red'], 
                                        index=states['density'].quantile([0.25,0.5,0.75]), 
                                        vmin=states['density'].min(), 
                                        vmax=states['density'].max()
                                       )

In [8]:
m = folium.Map(location=[38,-97], zoom_start=4)

stategeo = folium.GeoJson(states,
                          name='US States',
                          style_function=lambda x: {'fillColor':colormap(x['properties']['density']),'color':'black','weight':2},
                          tooltip=folium.GeoJsonTooltip(fields=['name','density'], 
                                            aliases=['Name','Density'], 
                                            localize=True)
                         ).add_to(m)

citygeo = folium.GeoJson(us_cities,
                         name='US Cities',
                         tooltip=folium.GeoJsonTooltip(fields=['nameascii'], 
                                                       labels=False)
                        ).add_to(m)


statesearch = Search(stategeo, 
                     geom_type='Polygon', 
                     placeholder="Search for a US State", 
                     collapsed=False, 
                     search_label='name', 
                     fill_color='red', 
                     weight=3
                    ).add_to(m)

citysearch = Search(citygeo, 
                    geom_type='Point', 
                    placeholder="Search for a US City", 
                    collapsed=True, 
                    search_label='nameascii'
                   ).add_to(m)

folium.LayerControl().add_to(m)

In [9]:
m